# **PAPER TRADING**

## **LIBRAIRIES**

In [ ]:
!pip install ib_insync
!pip install nest_asyncio
!pip install schedule
!pip install pytz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import schedule
from ib_insync import *
import yfinance as yf
import nest_asyncio
import pytz

## **API CONNEXION**

In [ ]:
# Interactive Broker API
# Connect to IB Gateway via ngrok's forwarding address
# ngrok config add-authtoken 2rw2xcnDr9K436GIMIPrJDYXrZk_4DYJZX5ic7odKvK27xU3s
# ngrok tcp 4002
# Replace '4.tcp.eu.ngrok.io' and '16234' with your ngrok forwarding details

# Apply patch for Colab's event loop issue
nest_asyncio.apply()

# Create an IB instance
ib = IB()

try:
    ib.connect('5.tcp.eu.ngrok.io', 12708, clientId=1)
    print("Connected to IB Gateway!")
except Exception as e:
    print(f"Failed to connect: {e}")

Connected to IB Gateway!


## **STRADDLE'S RETURNS (9:35-10:00)**

### **FUNCTIONS**

#### **STOCK PRICES**

In [ ]:
def get_sp500_tickers():
    """
    Fetch the list of S&P 500 tickers from Wikipedia.

    Returns:
        pd.DataFrame: DataFrame containing tickers and company names.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sp500_table = pd.read_html(url)[0]
    return sp500_table[["Symbol", "Security"]]


def fetch_intraday_stock_price_ib(ib, ticker, date):
    """
    Fetch the stock price for 5 minutes after the US market opens using the IB API.

    Args:
        ib (IB instance): The IBKR connection object.
        ticker (str): Stock ticker symbol (e.g., "AAPL").
        date (str): Date in 'YYYY-MM-DD' format.

    Returns:
        float: Stock price at 9:35 AM US Eastern Time or the last valid price before the target time.
    """
    # Parse the input date
    if isinstance(date, str):
        date = datetime.strptime(date, "%Y-%m-%d")

    # Define time zones
    us_eastern_tz = pytz.timezone("US/Eastern")
    utc_tz = pytz.UTC

    # Adjust for Daylight Saving Time (DST)
    market_open_time = datetime(date.year, date.month, date.day, 9, 35, 0, tzinfo=us_eastern_tz).astimezone(utc_tz)

    # Create the IB contract
    stock = Stock(ticker, "SMART", "USD")

    # Fetch historical data from IB
    try:
        # Ensure the contract is valid
        ib.qualifyContracts(stock)

        # Request historical data
        bars = ib.reqHistoricalData(
            stock,
            endDateTime=market_open_time,
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="TRADES",
            useRTH=True
        )

        # Convert bars to a DataFrame
        if bars:
            df = util.df(bars)
            df['timestamp'] = pd.to_datetime(df['date'])
            df.set_index('timestamp', inplace=True)

            # Filter for the target time (9:35 AM US Eastern Time)
            target_row = df[df.index == market_open_time]
            if not target_row.empty:
                return target_row.iloc[0]['close']
            else:
                # Return the last available price before the target time
                last_price_row = df[df.index < market_open_time]
                if not last_price_row.empty:
                    return last_price_row.iloc[-1]['close']
                else:
                    return 0  # No valid data
        else:
            print(f"No data found for {ticker} on {date.strftime('%Y-%m-%d')}.")
            return 0
    except Exception as e:
        print(f"Error fetching intraday stock price for {ticker}: {e}")
        return 0


#### **OPTION CHAINS**

In [ ]:
def fetch_option_chain(ib, symbol, stock_price, current_date):
    """
    Fetch the option chain for a given ticker symbol using IB API.

    Args:
        ib (IB instance): The IBKR connection object.
        symbol (str): The stock ticker symbol (e.g., "AAPL").
        stock_price (float): The current stock price.
        current_date (str): The current date in 'YYYY-MM-DD' format.

    Returns:
        dict: Filtered option chain with valid expirations and closest strike {"expirations":[exp], "closest_strike":strike}.
    """

    # Debugging: Print symbol to ensure it's correct
    print(f"Fetching option chain for: {symbol} at {current_date}, stock price: {stock_price}")

    # Qualify the stock contract
    try:
        contracts = ib.qualifyContracts(Stock(symbol, 'SMART', 'USD'))
        if not contracts:
            print(f"Error: No valid contract found for {symbol}")
            return None  # Return None to prevent IndexError
        stock = contracts[0]  # Retrieve the first qualified contract
    except Exception as e:
        print(f"Error qualifying contract for {symbol}: {e}")
        return None

    # Request the option chain
    try:
        option_chain = ib.reqSecDefOptParams(symbol, '', 'STK', stock.conId)
        if not option_chain:
            print(f"Error: No option chain data available for {symbol}")
            return None

        # Extract relevant data
        expirations = sorted(option_chain[0].expirations)
        strikes = sorted(option_chain[0].strikes)

        # Ensure current_date is a datetime object
        current_date = datetime.strptime(current_date, '%Y-%m-%d')

        # Filter expirations between 30 and 180 days
        filtered_expirations = [
            exp for exp in expirations
            if 30 <= (datetime.strptime(exp, '%Y%m%d') - current_date).days <= 180
        ]

        # Avoid fractional strikes to find contracts with IB
        valid_strikes = [s for s in strikes if s % 1 == 0]

        # Find the closest ATM strike
        closest_strike = min(valid_strikes, key=lambda x: abs(x - stock_price)) if valid_strikes else None

        return {'expirations': filtered_expirations, 'closest_strike': closest_strike}

    except Exception as e:
        print(f"Error fetching option chain for {symbol}: {e}")
        return None


#### **OPTION PRICES**

In [ ]:
def fetch_intraday_option_prices(ib, ticker, expiration, strike):
    """
    Fetch intraday option prices from IBKR at 9:35 AM and 10:00 AM US Eastern Time.

    Args:
        ib (IB instance): The IBKR connection object.
        ticker (str): Underlying stock ticker (e.g., 'MMM').
        expiration (str): Expiration date in 'YYYYMMDD' format (e.g., '20250228').
        strike (float): Strike price (e.g., 150.0).

    Returns:
        dict: { "5_min": [call price, put price], "30_min": [call price, put price] }
    """

    # **Format Strike Price Properly**
    formatted_strike = "{:.8f}".format(strike).rstrip("0").rstrip(".")

    # **Create IB Contracts for Call and Put**
    call_contract = Option(ticker, expiration, formatted_strike, 'C', 'SMART')
    put_contract = Option(ticker, expiration, formatted_strike, 'P', 'SMART')

    # **Request Historical Data**
    call_bars = ib.reqHistoricalData(call_contract, '', '1 D', '1 min', 'MIDPOINT', useRTH=True)
    put_bars = ib.reqHistoricalData(put_contract, '', '1 D', '1 min', 'MIDPOINT', useRTH=True)

    # **Convert Data to DataFrame**
    call_df = pd.DataFrame(call_bars) if call_bars else pd.DataFrame()
    put_df = pd.DataFrame(put_bars) if put_bars else pd.DataFrame()

    # **Check if Data Exists**
    if call_df.empty or put_df.empty:
        print(f"Error: No data retrieved for {ticker}, expiration {expiration}, strike {formatted_strike}")
        return None

    # **Find Closest Available Prices**
    call_price_5min = call_df.iloc[(call_df['date'] - call_df['date'].iloc[0].replace(hour=9, minute=35)).abs().argsort()[:1]]['close'].values[0] if not call_df.empty else None
    call_price_30min = call_df.iloc[(call_df['date'] - call_df['date'].iloc[0].replace(hour=10, minute=0)).abs().argsort()[:1]]['close'].values[0] if not call_df.empty else None
    put_price_5min = put_df.iloc[(put_df['date'] - put_df['date'].iloc[0].replace(hour=9, minute=35)).abs().argsort()[:1]]['close'].values[0] if not put_df.empty else None
    put_price_30min = put_df.iloc[(put_df['date'] - put_df['date'].iloc[0].replace(hour=10, minute=0)).abs().argsort()[:1]]['close'].values[0] if not put_df.empty else None

    return {
        "5_min": [call_price_5min, put_price_5min],
        "30_min": [call_price_30min, put_price_30min]
    }


#### **STRADDLE RETURNS**

In [ ]:
def calculate_straddle_returns(option_prices):
    """
    Calculate straddle return for each ticker.

    Args:
        option_prices (dict): Dictionary in the format
            {ticker_name: {5_min: [call_price, put_price], 30_min: [call_price, put_price]}}.

    Returns:
        dict: Dictionary in the format {ticker_name: straddle_return}.
    """
    straddle_returns = {}

    for ticker, prices in option_prices.items():
      if prices is not None:
        # Extract 5-minute and 30-minute prices
        open_prices = prices.get('5_min', [None, None])
        close_prices = prices.get('30_min', [None, None])

        # Ensure no missing data
        if None in open_prices or None in close_prices:
            straddle_returns[ticker] = None
            continue

        # Calculate the straddle prices
        straddle_open = sum(open_prices)
        straddle_close = sum(close_prices)

        # Ensure straddle_open is not zero to avoid division by zero
        if straddle_open == 0:
            straddle_returns[ticker] = straddle_close
        else:
            straddle_returns[ticker] = (straddle_close - straddle_open) / straddle_open
      else:
        straddle_returns[ticker] = None

    return straddle_returns

#### **RANK RETURNS**

In [ ]:
def rank_straddle_return(straddle_returns):
  """Rank tickers by straddle returns.

    Args:
        straddles_returns (dict): Dictionary in the format {ticker_name : straddle_return}.

    Returns:
        List : List of tuples in the format [(ticker_name, straddle_return)].
    """
  ranked_tickers = sorted(straddle_returns.items(), key=lambda x: x[1] if x[1] is not None else -float('inf'), reverse=True)

  return ranked_tickers


#### **OPTIONS CONTRACTS**

In [ ]:
def get_contracts_for_short_long(ranked_returns, options_chain):

  # Convert to DataFrame
  df = pd.DataFrame(ranked_returns, columns=['Ticker', 'Return'])

  # Drop rows where return is None
  df = df.dropna(subset=['Return'])

  # Compute quantiles
  q1 = df['Return'].quantile(0.20)
  q5 = df['Return'].quantile(0.80)

  # Select tickers in Q1 and Q5
  q1_tickers = df[df['Return'] <= q1]['Ticker'].tolist()
  q5_tickers = df[df['Return'] >= q5]['Ticker'].tolist()

  # Contract options to short
  q1_contracts = {}
  for ticker in q1_tickers:
    expiration = options_chain[ticker]["expirations"][0]
    strike = options_chain[ticker]["closest_strike"]
    q1_contracts[ticker] = {
        'expiration':expiration,
        'strike': strike,
    }

  # Contract options to buy
  q5_contracts = {}
  for ticker in q5_tickers:
    expiration = options_chain[ticker]["expirations"][0]
    strike = options_chain[ticker]["closest_strike"]
    q5_contracts[ticker] = {
        'expiration':expiration,
        'strike': strike,
    }

  return q1_contracts, q5_contracts

### **EXECUTION**

In [ ]:
# Set date of last day
date = (datetime.today()-timedelta(days=1)).strftime("%Y-%m-%d")

In [ ]:
# Get tickers of sp500
sp500_tickers = get_sp500_tickers()["Symbol"].tolist()

In [ ]:
# Get current stock prices
sp500_stock_prices = {}
for ticker in sp500_tickers:
  sp500_stock_prices[ticker] = fetch_intraday_stock_price_ib(ib, ticker, date)

ERROR:ib_insync.wrapper:Error 200, reqId 804: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Stock(symbol='BRK.B', exchange='SMART', currency='USD')
ERROR:ib_insync.wrapper:Error 200, reqId 805: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Stock(symbol='BRK.B', exchange='SMART', currency='USD')


No data found for BRK.B on 2025-02-09.


ERROR:ib_insync.wrapper:Error 200, reqId 834: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Stock(symbol='BF.B', exchange='SMART', currency='USD')
ERROR:ib_insync.wrapper:Error 200, reqId 835: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Stock(symbol='BF.B', exchange='SMART', currency='USD')


No data found for BF.B on 2025-02-09.


In [ ]:
# Get strike and expirations based on conditions (ATM and 30days<exp<180days)
sp500_option_chains = {}
for ticker, price in sp500_stock_prices.items():
  sp500_option_chains[ticker] = fetch_option_chain(ib, ticker, price, date)

Fetching option chain for: MMM at 2025-02-09, stock price: 149.9
Fetching option chain for: AOS at 2025-02-09, stock price: 65.18
Fetching option chain for: ABT at 2025-02-09, stock price: 129.08
Fetching option chain for: ABBV at 2025-02-09, stock price: 190.6
Fetching option chain for: ACN at 2025-02-09, stock price: 386.03
Fetching option chain for: ADBE at 2025-02-09, stock price: 433.05
Fetching option chain for: AMD at 2025-02-09, stock price: 107.52
Fetching option chain for: AES at 2025-02-09, stock price: 10.57
Fetching option chain for: AFL at 2025-02-09, stock price: 103.57
Fetching option chain for: A at 2025-02-09, stock price: 145.26
Fetching option chain for: APD at 2025-02-09, stock price: 310.42
Fetching option chain for: ABNB at 2025-02-09, stock price: 134.99
Fetching option chain for: AKAM at 2025-02-09, stock price: 99.64
Fetching option chain for: ALB at 2025-02-09, stock price: 77.36
Fetching option chain for: ARE at 2025-02-09, stock price: 95.96
Fetching option

ERROR:ib_insync.wrapper:Error 200, reqId 1810: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Stock(symbol='BRK.B', exchange='SMART', currency='USD')


Error: No valid contract found for BRK.B
Fetching option chain for: BBY at 2025-02-09, stock price: 84.87
Fetching option chain for: TECH at 2025-02-09, stock price: 72.82
Fetching option chain for: BIIB at 2025-02-09, stock price: 141.4
Fetching option chain for: BLK at 2025-02-09, stock price: 992.43
Fetching option chain for: BX at 2025-02-09, stock price: 170.25
Fetching option chain for: BK at 2025-02-09, stock price: 86.14
Fetching option chain for: BA at 2025-02-09, stock price: 181.4
Fetching option chain for: BKNG at 2025-02-09, stock price: 4887.52
Fetching option chain for: BWA at 2025-02-09, stock price: 30.01
Fetching option chain for: BSX at 2025-02-09, stock price: 105.28
Fetching option chain for: BMY at 2025-02-09, stock price: 56.85
Fetching option chain for: AVGO at 2025-02-09, stock price: 224.87
Fetching option chain for: BR at 2025-02-09, stock price: 240.48
Fetching option chain for: BRO at 2025-02-09, stock price: 108.58
Fetching option chain for: BF.B at 2025-0

ERROR:ib_insync.wrapper:Error 200, reqId 1839: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Stock(symbol='BF.B', exchange='SMART', currency='USD')


Error: No valid contract found for BF.B
Fetching option chain for: BLDR at 2025-02-09, stock price: 153.99
Fetching option chain for: BG at 2025-02-09, stock price: 69.62
Fetching option chain for: BXP at 2025-02-09, stock price: 72.76
Fetching option chain for: CHRW at 2025-02-09, stock price: 96.6
Fetching option chain for: CDNS at 2025-02-09, stock price: 299.71
Fetching option chain for: CZR at 2025-02-09, stock price: 35.74
Fetching option chain for: CPT at 2025-02-09, stock price: 119.17
Fetching option chain for: CPB at 2025-02-09, stock price: 37.54
Fetching option chain for: COF at 2025-02-09, stock price: 203.13
Fetching option chain for: CAH at 2025-02-09, stock price: 127.43
Fetching option chain for: KMX at 2025-02-09, stock price: 85.22
Fetching option chain for: CCL at 2025-02-09, stock price: 26.72
Fetching option chain for: CARR at 2025-02-09, stock price: 64.17
Fetching option chain for: CAT at 2025-02-09, stock price: 363.75
Fetching option chain for: CBOE at 2025-02

In [ ]:
# Fetch options price
sp500_option_prices = {}
for ticker, chain in sp500_option_chains.items():
    # Check if the option chain is valid
    if not chain or "expirations" not in chain or not chain["expirations"]:
        print(f"Skipping {ticker}: No valid option chain data")
        continue

    expiration = chain["expirations"][0]
    closest_strike = chain["closest_strike"]

    if closest_strike is None:
        print(f"Skipping {ticker}: No valid strike price found")
        continue

    sp500_option_prices[ticker] = fetch_intraday_option_prices(
        ib, ticker, expiration, closest_strike
    )

ERROR:ib_insync.wrapper:Error 200, reqId 2796: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Option(symbol='AZO', lastTradeDateOrContractMonth='20250321', strike='3430', right='C', exchange='SMART')
ERROR:ib_insync.wrapper:Error 200, reqId 2797: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Option(symbol='AZO', lastTradeDateOrContractMonth='20250321', strike='3430', right='P', exchange='SMART')


Error: No data retrieved for AZO, expiration 20250321, strike 3430
Skipping BRK.B: No valid option chain data


ERROR:ib_insync.wrapper:Error 200, reqId 2818: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Option(symbol='BIIB', lastTradeDateOrContractMonth='20250314', strike='141', right='C', exchange='SMART')
ERROR:ib_insync.wrapper:Error 200, reqId 2819: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Option(symbol='BIIB', lastTradeDateOrContractMonth='20250314', strike='141', right='P', exchange='SMART')


Error: No data retrieved for BIIB, expiration 20250314, strike 141
Skipping BF.B: No valid option chain data


ERROR:ib_insync.wrapper:Error 200, reqId 3020: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Option(symbol='EA', lastTradeDateOrContractMonth='20250321', strike='129', right='C', exchange='SMART')
ERROR:ib_insync.wrapper:Error 200, reqId 3021: Aucune d\u00e9finition de titre trouv\u00e9e pour cette requ\u00eate, contract: Option(symbol='EA', lastTradeDateOrContractMonth='20250321', strike='129', right='P', exchange='SMART')


...

ERROR:ib_insync.wrapper:Error 162, reqId 3320: Message d'erreur Service Donn\u00e9es de March\u00e9 Historiques:La demande HMDS n'a fourni aucune donn\u00e9e\u00a0: MGM   250314C00035000@SMART Midpoint, contract: Option(symbol='MGM', lastTradeDateOrContractMonth='20250314', strike='35', right='C', exchange='SMART')
ERROR:ib_insync.wrapper:Error 162, reqId 3321: Message d'erreur Service Donn\u00e9es de March\u00e9 Historiques:La demande HMDS n'a fourni aucune donn\u00e9e\u00a0: MGM   250314P00035000@SMART Midpoint, contract: Option(symbol='MGM', lastTradeDateOrContractMonth='20250314', strike='35', right='P', exchange='SMART')


Error: No data retrieved for MGM, expiration 20250314, strike 35


ERROR:ib_insync.wrapper:Error 162, reqId 3322: Message d'erreur Service Donn\u00e9es de March\u00e9 Historiques:La demande HMDS n'a fourni aucune donn\u00e9e\u00a0: MCHP  250314C00052000@SMART Midpoint, contract: Option(symbol='MCHP', lastTradeDateOrContractMonth='20250314', strike='52', right='C', exchange='SMART')
ERROR:ib_insync.wrapper:Error 162, reqId 3323: Message d'erreur Service Donn\u00e9es de March\u00e9 Historiques:La demande HMDS n'a fourni aucune donn\u00e9e\u00a0: MCHP  250314P00052000@SMART Midpoint, contract: Option(symbol='MCHP', lastTradeDateOrContractMonth='20250314', strike='52', right='P', exchange='SMART')


Error: No data retrieved for MCHP, expiration 20250314, strike 52


KeyboardInterrupt: 

In [ ]:
# Calculate straddle returns
straddle_returns = calculate_straddle_returns(sp500_option_prices)

In [ ]:
# Rank straddle returns
ranked_returns = rank_straddle_return(straddle_returns)

In [ ]:
# Get contracts for the next day
short_contracts, long_contracts = get_contracts_for_short_long(ranked_returns, sp500_option_chains)


## **MANAGE TRADES**

### **FUNCTIONS**

#### **OPTIONS CONTRACTS**

In [ ]:
def generate_options_contracts(ib, contracts):
    """
    Open (BUY or SELL) straddle positions for the given tickers.

    Args:
        ib (IB instance): The IBKR connection object.
        contracts (pd.DataFrame): DataFrame of straddles to trade in the format:
                          col = ticker, rows = ['expiration', 'strike']
    Returns:
       pd.DataFrame: DataFrame of straddles to trade in the format:
                          col = ticker, rows = ['call', 'put'], value = Option Contract.
    """

    # Contract options
    options_contracts = {}
    for ticker in contracts.keys():
      expiration = contracts[ticker]["expiration"]
      atm_strike = contracts[ticker]["strike"]
      options_contracts[ticker] = {
          'call': Option(symbol=ticker, lastTradeDateOrContractMonth=expiration, strike=atm_strike, right='C', exchange='SMART'),
          'put': Option(symbol=ticker, lastTradeDateOrContractMonth=expiration, strike=atm_strike, right='P', exchange='SMART'),
      }

    return pd.DataFrame.from_dict(options_contracts)

#### **OPEN POSITIONS**

In [ ]:
def open_straddle_positions(ib, straddles, action):
    """
    Open (BUY or SELL) straddle positions for the given tickers.

    Args:
        ib (IB instance): The IBKR connection object.
        straddles (pd.DataFrame): DataFrame of straddles to trade in the format:
                          col = ticker, rows = ['call', 'put'], value = Option Contract.
        action (str): 'BUY' for long straddle, 'SELL' for short straddle.
    """
    if action not in ['BUY', 'SELL']:
        raise ValueError("Action must be 'BUY' or 'SELL'.")

    for ticker in straddles.keys():
        call_contract = straddles.loc['call', ticker]
        put_contract = straddles.loc['put', ticker]

        # Ensure contracts are valid
        ib.qualifyContracts(call_contract, put_contract)

        # Place market orders to buy the call and the put
        call_order = MarketOrder(action, 1)
        put_order = MarketOrder(action, 1)

        # Submit orders
        ib.placeOrder(call_contract, call_order)
        ib.placeOrder(put_contract, put_order)

        print(f"Opened straddle for {ticker}: Call and Put orders placed.")


#### **CLOSE ALL POSITIONS**

In [ ]:
def close_all_positions(ib):
    """
    Close all open positions in the account.
    """
    positions = ib.positions()
    for position in positions:
        contract = position.contract
        quantity = position.position

        # Close the position by placing a market order in the opposite direction
        if quantity > 0:
            close_order = MarketOrder('SELL', quantity)
        else:
            close_order = MarketOrder('BUY', abs(quantity))

        ib.placeOrder(contract, close_order)
        print(f"Closed position for {contract.localSymbol}.")


### **EXECUTION**

In [ ]:
# Generate options objects
long_straddle = generate_options_contracts(ib, long_contracts)
short_straddle = generate_options_contracts(ib, short_contracts)

In [ ]:
# Open short and long positions 5 minutes after the openning (9:35 US MARKET HOURS)
open_straddle_positions(ib, long_straddle, 'BUY')
print("All long straddles placed.")
open_straddle_positions(ib, short_straddle, 'SELL')
print("All short straddles opened.")

Opened straddle for T: Call and Put orders placed.
Opened straddle for HPE: Call and Put orders placed.
Opened straddle for FE: Call and Put orders placed.
Opened straddle for BMY: Call and Put orders placed.
Opened straddle for CSX: Call and Put orders placed.
Opened straddle for BALL: Call and Put orders placed.
Opened straddle for FOXA: Call and Put orders placed.
Opened straddle for EQT: Call and Put orders placed.
Opened straddle for GEN: Call and Put orders placed.
Opened straddle for HAS: Call and Put orders placed.
Opened straddle for FCX: Call and Put orders placed.
Opened straddle for CTVA: Call and Put orders placed.
Opened straddle for CSCO: Call and Put orders placed.
Opened straddle for CNC: Call and Put orders placed.
Opened straddle for CZR: Call and Put orders placed.
Opened straddle for FOX: Call and Put orders placed.
Opened straddle for CTSH: Call and Put orders placed.
Opened straddle for DAL: Call and Put orders placed.
Opened straddle for CCL: Call and Put orders

In [ ]:
# Close all positions 25 minutes after openning trades (10:00 US MARKET HOURS)
close_all_positions(ib)

Closed position for ALLE  250321C00130000.
Closed position for ARE   250321C00100000.
Closed position for T     250307P00024000.
Closed position for AEE   250321P00095000.
Closed position for BX    250307C00190000.
Closed position for APH   250321P00070000.
Closed position for AMGN  250228C00280000.
Closed position for CPT   250321C00110000.
Closed position for ADP   250228C00305000.
Closed position for BK    250307P00087000.
Closed position for BK    250228P00086000.
Closed position for TECH  250321C00075000.
Closed position for MO    250228P00053000.
Closed position for ABT   250228P00128000.
Closed position for SCHW  250307P00082000.
Closed position for AFL   250228C00107000.
Closed position for CNC   250228P00065000.
Closed position for ATO   250321C00140000.
Closed position for BDX   250321P00250000.
Closed position for BBY   250228C00087000.
Closed position for AFL   250228P00107000.
Closed position for ADM   250228P00051000.
Closed position for BMY   250307C00059000.
Closed posi

## **MANAGE PNL**

### **FUNCTIONS**

#### **REAL TIME PNL (ACCOUNT LEVEL)**

In [ ]:
def get_account_pnl(ib):
    """
    Fetch the realized and unrealized PnL from Interactive Brokers.

    Args:
        ib (IB instance): The IBKR connection object.

    Returns:
        float: realized_pnl
    """
    # Fetch account summary (returns a list)
    account_summary = ib.accountSummary()

    # Convert list to DataFrame
    df = pd.DataFrame([(x.tag, x.value) for x in account_summary], columns=['tag', 'value'])

    # Convert values to float, handling possible missing values
    realized_pnl = float(df.loc[df['tag'] == 'NetLiquidation', 'value'].values[0]) if 'NetLiquidation' in df['tag'].values else None

    return realized_pnl

#### **REAL TIME PNL (TICKER LEVEL)**

In [ ]:
def get_portfolio_pnl(ib):
    """
    Fetch PnL for each individual position in the portfolio, including open & close dates.
    """
    portfolio = ib.portfolio()  # Get portfolio positions
    trades = ib.trades()  # Fetch all trades
    pnl_data = []

    for position in portfolio:
        contract = position.contract
        pnl = position.unrealizedPNL + position.realizedPNL

        # Use correct attribute name and handle missing values safely
        avg_price = getattr(position, 'averageCost', None)

        # Get executions for this contract
        trade_exec_times = [trade.log[0].time for trade in trades if trade.contract == contract]

        # Open Date = First execution time (when the trade was first opened)
        open_date = min(trade_exec_times) if trade_exec_times else None

        pnl_data.append({
            'Ticker': contract.localSymbol,
            'Quantity': position.position,
            'Avg Price': avg_price,
            'PnL': pnl,
            'Open Date': open_date
        })

    df_pnl = pd.DataFrame(pnl_data)

    return df_pnl

### **EXECUTION**

In [ ]:
# Account pnl
realized_pnl = get_account_pnl(ib)
realized_pnl

979290.09

In [ ]:
# Positions pnl
df_pnl = get_portfolio_pnl(ib)
df_pnl

,Ticker,Quantity,Avg Price,PnL,Open Date
0,A 250321C00150000,-3.0,626.511200,1039.53,NaT
1,A 250321P00150000,-3.0,523.180733,-1201.88,NaT
2,AAPL 250307C00240000,-1.0,714.402100,537.95,NaT
3,AAPL 250307P00240000,-1.0,604.405100,-573.78,NaT
4,ABBV 250307C00175000,1.0,1580.575300,76.46,NaT
...,...,...,...,...,...
321,AVY 250321P00180000,-1.0,438.079728,-83.31,2025-02-10 14:36:29.653351+00:00
322,FTNT 250314C00108000,-1.0,450.129422,-64.34,2025-02-10 14:36:33.465747+00:00
323,BLDR 250321P00155000,-1.0,839.838608,-6.62,2025-02-10 14:36:12.549263+00:00
324,FOXA 250314C00053000,1.0,185.135250,-72.51,2025-02-10 14:35:25.907764+00:00
